In [ ]:
%matplotlib qt5
import tensorflow as tf
import numpy as np
from collections import deque
import random
import matplotlib.pyplot as plt
class Q_Network:
    N_Mesh=15
    State_Number=N_Mesh*N_Mesh;
    Action_Number=4;
    Action_List=np.identity(Action_Number)
    State_List=np.identity(State_Number)
    Epsilon=0
    Epsilon_Begin=1
    Epsilon_Final=0.01
    decay_rate=0.000001
    Step=0
    Batch_Number=30
    #Explore = 100000.
    Observe=1000#The number of steps of observation before the beginning of the training
    Store_Memory = deque()
    Cost_History=[]
    Memory_Size = 5000
    Obstacle=[3,5,16,21,23,27,29,31,33,34,56,57,66,67,78,81,86,87,88,89,103,107,110,118,121,132,133,134,135,146,149,153
              ,154,155,156,157,176,180]
    Play_Done=None
    def __init__(self,Learning_Rate=0.001,Gamma=0.9,Memory_Size=5000):
        self.Learning_Rate=Learning_Rate
        self.Gamma=Gamma
        self.Memory_Size=Memory_Size
        self.New_Network()
        self.session=tf.InteractiveSession()
        self.session.run(tf.initialize_all_variables())
     #***************************************************************
    def New_Network(self):
         #Construct a neural network
        self.State_Input=tf.placeholder(tf.float32,[None,self.State_Number])#Input_Number=100, M=Batch*100
        self.Action_Input=tf.placeholder(tf.float32,[None,self.Action_Number])#Input_Number=4, M=Batch*4
        self.Q_Target_OI=tf.placeholder(dtype=tf.float32,shape=[None])
        #Layer1
        Layer1_Number=200
        w1=tf.Variable(tf.random_normal([self.State_Number,Layer1_Number]))
        b1=tf.Variable(tf.zeros([1,Layer1_Number])+0.1)
        l1=tf.nn.relu(tf.matmul(self.State_Input,w1)+b1) #b1 use the propagation mechanism
        #Layer2
        Layer2_Number=40
        w2=tf.Variable(tf.random_normal([Layer1_Number,Layer2_Number]))
        b2=tf.Variable(tf.zeros([1,Layer2_Number]))
        l2=tf.nn.relu(tf.matmul(l1,w2)+b2)
        #Layer3
        Layer3_Number=self.Action_Number
        w3=tf.Variable(tf.random_normal([Layer2_Number,Layer3_Number]))
        b3=tf.Variable(tf.zeros([1,Layer3_Number]))
        self.l3=tf.matmul(l2,w3)+b3#At this time, not yet use the active action, l3 is the output matrice, M=Batch*4
        #self.l3=tf.tanh(tf.matmul(l1,w2)+b2)

        self.Q_Value=tf.reduce_sum(tf.multiply(self.l3,self.Action_Input),reduction_indices=1)
        self.Loss=tf.reduce_mean(tf.square(self.Q_Value-self.Q_Target_OI))
        #self.Loss=tf.reduce_mean(tf.square(self.Q_Value-tf.tanh(self.Q_Target_OI)))
        self.Optimizer=tf.train.AdamOptimizer(self.Learning_Rate).minimize(self.Loss)
        #Network's Output is like this form [19,20,297,30], argmax is 2, then choose Action2  
        self.Predict=tf.argmax(self.l3,1)
     #***************************************************************
    def Select_Action(self,State_Index):
        Current_State=self.State_List[State_Index:State_Index+1] #By example, Action_Index is 2, Current_State=[0,0,1,0， ，]
        if np.random.uniform()<self.Epsilon:
            Choose_Action_Index=np.random.randint(0,self.Action_Number)
        else:
            Action_QValue_Output=self.session.run(self.l3,feed_dict={self.State_Input:Current_State})
            Choose_Action_Index=np.argmax(Action_QValue_Output)
        #The first inegality means the train has beginned
        #The second inegality means Epsilon has not yet decreased to final value
        self.Epsilon=self.Epsilon_Final+(self.Epsilon_Begin-self.Epsilon_Final)*np.exp(-self.decay_rate*self.Step)
#         if(self.Step%1000==0):
#             print("epsilon ",self.Epsilon)
        return Choose_Action_Index
     #***************************************************************
    def Next_State(self,Action_Index,State_Index):
        State=State_Index+1
        done=False
        #Execute left motion
        if(Action_Index==0):
            if(State%self.N_Mesh==1):
                State=State
                R=-500
            elif((State+1) in self.Obstacle):#When encounter obstables, we stay 
                State=State
                R=-500
            else:
                State=State-1
                R=0
        elif(Action_Index==1):
            if(State==self.State_Number-1):
                State=State+1
                done=True
                R=100
            elif(State%self.N_Mesh==0):
                State=State
                R=-500
            elif((State+1) in self.Obstacle):
                State=State
                R=-500
            else:
                State=State+1
                R=0.2
        elif(Action_Index==2):
            if(State==self.State_Number-self.N_Mesh):
                State=State+self.N_Mesh
                done=True
                R=100
            elif(self.State_Number+1-self.N_Mesh<=State<=self.State_Number-1):
                State=State
                R=-500
            elif((State+self.N_Mesh) in self.Obstacle):
                State=State+self.N_Mesh
                R=-500
            else:
                State=State+self.N_Mesh
                R=0.2
        else:
            if(0<=State<=self.N_Mesh):
                State=State
                R=-500
            elif((State-self.N_Mesh) in self.Obstacle):
                State=State-self.N_Mesh
                R=-500
            else:
                R=0
                State=State-self.N_Mesh
        return State-1,R,done
    
    def Save_Memory(self,CURRENT_STATET,CHOOSE_ACTION,NEXT_STATE,REWARD,DONE):
        Memory_Current_State=self.State_List[CURRENT_STATET:CURRENT_STATET+1]
        Memory_Choose_Action=self.Action_List[CHOOSE_ACTION:CHOOSE_ACTION+1]
        Memory_Next_State=self.State_List[NEXT_STATE:NEXT_STATE+1]
        self.Store_Memory.append((Memory_Current_State,Memory_Choose_Action,Memory_Next_State,REWARD,DONE))
        if len(self.Store_Memory)>self.Memory_Size:
            self.Store_Memory.popleft()
    
     #***************************************************************
    def Experience_Replay(self):
        Batch=self.Batch_Number
        MiniBatch=random.sample(self.Store_Memory,Batch)
        Batch_Current_State = None
        Batch_Execute_Action = None
        Batch_Reward = None
        Batch_Next_State = None
        Batch_Done = None
        
        for Index in range(Batch):
            if Batch_Current_State is None:
                Batch_Current_State=MiniBatch[Index][0]
            elif Batch_Current_State is not None:
                Batch_Current_State=np.vstack((Batch_Current_State, MiniBatch[Index][0]))
            #---------------------------------------------------------------------------------------   
            #print("BEA",MiniBatch[Index][1].shape)
            if Batch_Execute_Action is None:
                Batch_Execute_Action=MiniBatch[Index][1]
            elif Batch_Execute_Action is not None:
                Batch_Execute_Action=np.vstack((Batch_Execute_Action,MiniBatch[Index][1]))
           #---------------------------------------------------------------------------------------           
            if Batch_Reward is None:
                Batch_Reward=MiniBatch[Index][3]
            elif Batch_Reward is not None:
                Batch_Reward=np.vstack((Batch_Reward,MiniBatch[Index][3]))
            #---------------------------------------------------------------------------------------  
            if Batch_Next_State is None:
                Batch_Next_State=MiniBatch[Index][2]
            elif Batch_Next_State is not None:
                Batch_Next_State=np.vstack((Batch_Next_State,MiniBatch[Index][2]))
            #---------------------------------------------------------------------------------------      
            if Batch_Done is None:
                Batch_Done=MiniBatch[Index][4]
            elif Batch_Done is not None:
                Batch_Done=np.vstack((Batch_Done,MiniBatch[Index][4]))
            #Calculate the Q Value of the next State 
        Q_Next=self.session.run(self.l3,feed_dict={self.State_Input:Batch_Next_State})
        Q_Target=[]
        for i in range(Batch):
            Each_Reward=Batch_Reward[i][0]#This is a 2D array because of the vstack
            #Calculate the Target-Q-Value of each element in the Batch
            Each_QValue=Each_Reward+self.Gamma*np.max(Q_Next[i]) #The network ouput has its own []
            if Each_Reward<0:
                Q_Target.append(Each_Reward)
            else:
                Q_Target.append(Each_QValue)
        #print(self.session.run(self.Q_Value,feed_dict={self.Q_Target_OI:Q_Target}))
        _,Cost,Rew=self.session.run([self.Q_Value,self.Loss,self.Optimizer],feed_dict={self.State_Input:Batch_Current_State,
                                                                                        self.Action_Input:Batch_Execute_Action,
                                                                                        self.Q_Target_OI: Q_Target})
        self.Cost_History.append(Cost)
        if self.Step%5000==0:
            print(self.Step)  
            print("Loss:", Cost)  
            print("Epsilon:", self.Epsilon)   
    #**************************************************************
    def Train(self):
        Train_Current_State=np.random.randint(0,self.State_Number-1)
        self.Epsilon = self.Epsilon_Begin
        while True:
            Train_Action=self.Select_Action(Train_Current_State)
            #print("TA",Train_Action)
            Train_Next_State,Train_Reward,Train_Done=self.Next_State(Train_Action,Train_Current_State)
            self.Save_Memory(Train_Current_State,Train_Action,Train_Next_State,Train_Reward,Train_Done)
            if self.Step>self.Observe:
                self.Experience_Replay()
            if self.Step>2000000:
                 break;
            if Train_Done:
                Train_Current_State=np.random.randint(0,self.State_Number-1)
            else:
                Train_Current_State=Train_Next_State
                self.Step+=1          
    #***************************************************************
    def Play(self):
        self.Train()
        Start_Room_Index=0;
        Play_Current_State_Index=0
        Play_Step=0;
        print("****************** Agent is in the State",Start_Room_Index,"**********************")
        while(Play_Current_State_Index!=self.State_Number-1):
            Play_Current_State=self.State_List[Play_Current_State_Index:Play_Current_State_Index+1]
            Play_Action=(self.session.run(self.Predict,feed_dict={self.State_Input:Play_Current_State}))
            if Play_Action==0:
                Play_Current_State_Index=Play_Current_State_Index-1
            elif Play_Action==1:
                Play_Current_State_Index=Play_Current_State_Index+1
            elif Play_Action==2:
                Play_Current_State_Index=Play_Current_State_Index+self.N_Mesh
            else:
                Play_Current_State_Index=Play_Current_State_Index-self.N_Mesh
            Play_Step+=1
            print("Step is ",Play_Step,"****************** Agent is in the State", Play_Current_State_Index,"**********************")
            if(Play_Step>30):
                break
        if Play_Current_State_Index==self.State_Number-1:
            self.Graph()
    #***************************************************************        
    def Caculate_coordinate(self,State):
        X_Axis=0.5+(State-1)%self.N_Mesh
        Y_Axis=0.5+((State-1)//self.N_Mesh)
        return X_Axis,Y_Axis 
    #***************************************************************  
    def Graph(self): 
        Graph_Step=0
        fig=plt.figure()
        ax=fig.gca()
        ax.set(xlim=[0, self.N_Mesh], ylim=[0, self.N_Mesh])
        ax.set_xticks(np.arange(0,(self.N_Mesh+1)))
        ax.set_yticks(np.arange(0,(self.N_Mesh+1)))
        plt.grid()
        Graph_Start_Index=0
        Graph_Current_Index=Graph_Start_Index
        while(Graph_Current_Index!=self.State_Number-1):
            Graph_Current_State=self.State_List[Graph_Current_Index:Graph_Current_Index+1]
            Graph_Action=(self.session.run(self.Predict,feed_dict={self.State_Input:Graph_Current_State}))
            if(Graph_Action==0):
                Graph_Next_Index=Graph_Current_Index-1
            elif(Graph_Action==1):
                Graph_Next_Index=Graph_Current_Index+1
            elif(Graph_Action==2):
                 Graph_Next_Index=Graph_Current_Index+self.N_Mesh
            else:
                 Graph_Next_Index=Graph_Current_Index-self.N_Mesh
            Graph_Step+=1
            X_State,Y_State=self.Caculate_coordinate(Graph_Current_Index+1)
            X_Next_State,Y_Next_State=self.Caculate_coordinate(Graph_Next_Index+1)
            if(Graph_Step==0):
                plt.scatter(X_State,Y_State)  
            else:
                plt.scatter(X_State,Y_State) 
                plt.scatter(X_Next_State,Y_Next_State)
                plt.plot([X_State,X_Next_State],[Y_State,Y_Next_State])
            for J in np.arange(len(self.Obstacle)):
                plt.scatter((self.Caculate_coordinate(self.Obstacle[J]+1))[0],(self.Caculate_coordinate(self.Obstacle[J]+1))[1],marker="x")
            Graph_Current_Index=Graph_Next_Index
    plt.show()

if __name__ == "__main__":
    q_network = Q_Network()
    q_network.Play()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
5000
Loss: 47922.0
Epsilon: 0.995062354401
10000
Loss: 13964.7
Epsilon: 0.990149335412
15000
Loss: 7638.14
Epsilon: 0.985260820207
20000
Loss: 8168.88
Epsilon: 0.980396686574
25000
Loss: 6702.18
Epsilon: 0.975556812908
30000
Loss: 4179.47
Epsilon: 0.970741078213
35000
Loss: 5849.48
Epsilon: 0.965949362095
40000
Loss: 3407.37
Epsilon: 0.961181544761
45000
Loss: 4370.87
Epsilon: 0.956437507015
50000
Loss: 4264.85
Epsilon: 0.951717130256
55000
Loss: 5200.89
Epsilon: 0.947020296474
60000
Loss: 3527.57
Epsilon: 0.942346888248
65000
Loss: 5732.39
Epsilon: 0.937696788744
70000
Loss: 7409.68
Epsilon: 0.933069881707
75000
Loss: 3915.83
Epsilon: 0.928466051465
80000
Loss: 4021.38
Epsilon: 0.923885182923
85000
Loss: 5353.25
Epsilon: 0.919327161557
90000
Loss: 4880.43
Epsilon: 0.914791873419
95000
Loss: 5085.54
Epsilon: 0.910279205124
100000
Loss: 4462.06
Epsilon: 0.905789043856
105000
Loss: 3690.49
Epsilon: 0.90132127736
11